In [32]:
import numpy as np
template_first = '''&CPMD
  OPTIMIZE WAVEFUNCTION gEOMETRY
  kOHN-SHAM ENERGIES
   10
  PRINT ON FORCES
  MIRROR
  CENTER MOLECULE ON
  ELECTROSTATIC POTENTIAL
  MAXSTEP
  1
&END

&DFT
  FUNCTIONAL LDA
&END

&PROP
  DIPOLE MOMENT
&END

&SYSTEM
  ANGSTROM
  SYMMETRY
   ORTHORHOMBIC
  pOISSON SOLVER TUCKERMAN
  CUTOFF
   %f
  CELL ABSOLUT
        %f 1.0 1.0 0.0 0.0 0.0
  CHARGE
   %f
&END

&ATOMS
%s
&END'''
template_second= '''&CPMD
  OPTIMIZE WAVEFUNCTION gEOMETRY
  RESTART WAVEFUNCTION LATEST
  kOHN-SHAM ENERGIES
   10
  PRINT ON FORCES
  MIRROR
  RHOOUT
  CENTER MOLECULE OFF
  eLECTROSTATIC POTENTIAL
  mAXSTEP
  1
&END

&DFT
  FUNCTIONAL LDA
&END

&PROP
  DIPOLE MOMENT
&END

&SYSTEM
  ANGSTROM
  SYMMETRY
   ORTHORHOMBIC
  pOISSON SOLVER TUCKERMAN
  CUTOFF
   %f
  CELL ABSOLUT
   %f 1.0 1.0 0.0 0.0 0.0
  CHARGE
   %s
&END

&ATOMS
%s
&END'''

In [48]:
lookup = {1: 'H', 6: 'C', 7: 'N', 8: 'O'}
rlookup = {'C': 6, 'O': 8, 'N': 7}
def get_atoms_section(nuclear_charges, coordinates):
    nuclear_charges = np.array(nuclear_charges)
    coordinates = np.array(coordinates)
    ret = []
    for kind in set(nuclear_charges):
        ret.append('*%s_SG_LDA\n LMAX=S' % lookup[kind])
        idxs = np.where(nuclear_charges == kind)[0]
        ret.append(' %d' % len(idxs))
        for atom in idxs:
            ret.append(' %f %f %f' % (coordinates[atom, 0], coordinates[atom, 1], coordinates[atom, 2]))
    return '\n'.join(ret)
#print(get_atoms_section((6, 8), [[3, 4, 4], [5, 4, 4]]))
def get_PP_file(Z, lval):
    element = lookup[Z]
    fn = '/mnt/c/Users/guido/opt/cpmd/PP_LIBRARY/%s_SG_LDA' % element
    ret = []
    for line in open(fn).readlines():
        if 'ZV' in line:
            ZV = float(line.strip().split()[-1])
            ret.append('  ZV = %f' % (ZV*lval))
            continue
        if '#C' in line:
            parts = np.array([float(_) for _ in line.split('#')[0].strip().split()])
            parts[1:] *= lval
            formatstring = '%4d' + (len(parts)-1)*' %15.9f' + ' #C'
            ret.append(formatstring % (*parts,))
            continue
        ret.append(line[:-1])
    return ZV*(1-lval), '\n'.join(ret)
def center_coordinates(coords, box_length):
    cog = coords.mean(axis=0)
    cob = np.array([box_length/2.,box_length/2.,box_length/2.])
    return coords + (cob - cog)
def get_second_input(nuclear_charges, box_length, coordinates, cutoff, lval):
    coordinates = center_coordinates(np.array(coordinates), box_length)
    atoms = get_atoms_section(nuclear_charges, coordinates)
    files = {}
    charge = 0.
    for kind in set(nuclear_charges):
        element = lookup[kind]
        this_charge, files['%s_SG_LDA' % element] = get_PP_file(kind, lval)
        count = len([_ for _ in nuclear_charges if _ == kind])
        charge -= count*this_charge
        
    files['run.inp'] = template_second % (cutoff, box_length, charge, atoms)
    return files
def get_first_input(nuclear_charges, box_length, coordinates, cutoff):
    coordinates = center_coordinates(np.array(coordinates), box_length)
    atoms = get_atoms_section(nuclear_charges, coordinates)
    files = {}
    charge = 0.
    for kind in set(nuclear_charges):
        element = lookup[kind]
        this_charge, files['%s_SG_LDA' % element] = get_PP_file(kind, 1.0)
        count = len([_ for _ in nuclear_charges if _ == kind])
        charge -= count*this_charge
        
    files['run.inp'] = template_first % (cutoff, box_length, charge, atoms)
    return files
def read_xyz(fn):
    lines = open(fn).readlines()[2:]
    elements = []
    coords = []
    for line in lines:
        parts = line.strip().split()
        elements.append(rlookup[parts[0]])
        coords.append([float(_) for _ in parts[1:]])
    return elements, coords

In [49]:
#get_first_input((6, 8), 10, [[3, 4, 4], [5, 4, 4]], 200)
read_xyz('/mnt/c/Users/guido/workcopies/apdft/prototyping/cpmd-uniform-electron-gas/data/CO-2/inp.xyz')

([6, 8], [[3.0, 4.0, 4.0], [5.0, 4.0, 4.0]])

In [ ]:
# molecule/inp.xyzls

# molecule/box-10/init/
# molecule/box-10/lval-../